In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import DBSCAN
from itertools import product

In [ ]:
def generate_noise(width, length, amount, step, std=10, means=(1,5)):
        samples = np.zeros((width, length, amount, 2))

        clusters = np.random.randint(
            means[0], means[1] + 1, size=(width, length)
        )

        # calculate centers
        grid_width = (np.arange(width) + 1) * step
        grid_length = (np.arange(length) + 1) * step
        mean = np.array(
            [
                np.repeat(grid_width, len(grid_length)),
                np.tile(grid_length, len(grid_width)),
            ]
        ).T
        noise = np.random.randn(means[1], width * length, 2) * std
        centers = (noise + mean).reshape((means[1], width, length, 2))

        for i in range(width):
            for j in range(length):
                samples[i, j, :] = make_blobs(
                    n_samples=amount, centers=centers[0 : clusters[i, j], i, j, :]
                )[0]

        return samples

In [ ]:
np.random.seed(0)
data = generate_noise(3, 3, 50, 10)

In [ ]:
plt.plot(data[0,0,:,0], data[0,0,:,1], 'o') # example of 5 clusters in position 0,0
plt.show()

In [ ]:
def generate_map(noise, eps=2, min_samples=3):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(noise)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    return labels, core_samples_mask, n_clusters_

In [ ]:
def plot_clusters(X, labels, core_sapmles_mask, n_clusters_):
    
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=14)

        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6)
    plt.title('Estimated number of clusters: %d' % n_clusters_)

In [ ]:
for x,y in product(range(3), range(3)):
    labels, core_samples_mask, n_clusters_ = generate_map(data[x,y,:,:])
    plot_clusters(data[x,y,:,:], labels, core_samples_mask, n_clusters_)
    plt.show()